In [1]:
import pandas as pd
from faker import Faker
import random
from datetime import date, timedelta
import numpy as np
import string

fake = Faker('en_IN')
Faker.seed(42)
random.seed(42)

email_providers = [
    'gmail.com', 'hotmail.com', 'yahoo.com', 'hyderabad.bits-pilani.ac.in'
]

In [2]:
states = pd.read_csv("./res/states.csv", header=None).rename(columns={0: 'state', 1: 'name'})
states = states[['state', 'name']]
states.head()

,state,name
0,AP,Andhra Pradesh
1,AR,Arunachal Pradesh
2,AS,Assam
3,BR,Bihar
4,CG,Chattisgarh


In [3]:
cities = pd.read_csv('./res/cities.csv')[['state', 'district']]
cities

,state,district
0,AN,Nicobar
1,AN,North and Middle Andaman
2,AN,South Andaman
3,AP,Anantapur
4,AP,Chittoor
...,...,...
712,WB,Paschim Medinipur
713,WB,Purba Medinipur
714,WB,Purulia
715,WB,South 24 Parganas


In [4]:
cities = cities\
.merge(states, how='left', left_on='state', right_on='state')\
.dropna()[['district', 'name']]\
.rename(columns={'name': 'state'})

cities

,district,state
3,Anantapur,Andhra Pradesh
4,Chittoor,Andhra Pradesh
5,East Godavari,Andhra Pradesh
6,Guntur,Andhra Pradesh
7,Kadapa,Andhra Pradesh
...,...,...
712,Paschim Medinipur,West Bengal
713,Purba Medinipur,West Bengal
714,Purulia,West Bengal
715,South 24 Parganas,West Bengal


In [5]:
def get_random_city():
    return cities.to_numpy()[random.randint(0, len(cities)-1)]
get_random_city()

array(['Purba Bardhaman', 'West Bengal'], dtype=object)

In [6]:
locality = pd.read_csv('./res/localities.csv')[['key', 'place_name', 'admin_name1']]\
.rename(columns={'key': 'pincode', 'admin_name1': 'state'})
locality

,pincode,place_name,state
0,110001,Connaught Place,New Delhi
1,110002,Darya Ganj,New Delhi
2,110003,Aliganj,New Delhi
3,110004,Rashtrapati Bhawan,New Delhi
4,110005,Lower Camp Anand Parbat,New Delhi
...,...,...,...
11037,855116,Thakurganj,Bihar
11038,855117,Taiyabpur,Bihar
11039,855119,Terhagachi,Bihar
11040,855121,Balarampur,Bihar


In [7]:
def get_random_locality(state):
    df = locality.query('@state == state')
    if(len(df) == 0):
        df = locality
    return df.to_numpy()[min(random.randint(0, len(df) - 1), 5)]

get_random_locality('New Delhi')

array([110006, 'Bara Tooti', 'New Delhi'], dtype=object)

In [8]:
def rand_pass(size):
    generate_pass = ''.join([random.choice(
                        string.ascii_letters + string.digits)
                        for n in range(size)])
                          
    return generate_pass

In [9]:
def get_person():
    aadhar = random.randint(2000, 5000)
    age = random.randint(18, 60)
    firstname = fake.first_name()
    lastname = fake.last_name()
    email = firstname.lower() + '.' + lastname.lower() + str(
        random.randint(100, 2000)) + '@' + random.choice(email_providers)
    password = rand_pass(10)
    door = fake.building_number()
    cityObj = get_random_city()
    city = cityObj[0]
    state = cityObj[1]
    locality = get_random_locality(state)
    pin = locality[0]
    street = locality[1]
    
    query = "exec CreateNewUser('{}', {}, '{}', '{}', '{}', '{}', '{}', '{}', '{}', {});".format(
    aadhar, age, email, password, firstname + ' ' + lastname, door, street, city,
    state, pin)
    return (query, aadhar)


In [10]:
aadhars = []

with open('users.sql', 'w+') as f:
    for _ in range(20):
        person = get_person()
        aadhars.append(person[1])
        print(person[0])
        f.writelines(person[0] + '\n')

exec CreateNewUser('2102', 35, 'navya.bhalla601@hotmail.com', 'iVgRV5IfLB', 'Navya Bhalla', 'H.No. 433', 'Khowangghat', 'Upper Subansiri', 'Arunachal Pradesh', 785676);
exec CreateNewUser('2383', 31, 'hiran.shan576@gmail.com', 'JmTPSIAoCL', 'Hiran Shan', 'H.No. 819', 'Khalwa', 'Dewas', 'Madhya Pradesh', 450117);
exec CreateNewUser('2026', 28, 'aarna.babu1529@hyderabad.bits-pilani.ac.in', 'vrjn9Wvgfy', 'Aarna Babu', 'H.No. 133', 'Mahendru', 'Sheikhpura', 'Bihar', 800006);
exec CreateNewUser('3408', 56, 'dishani.ramachandran641@gmail.com', 'UDIh7yfJs1', 'Dishani Ramachandran', 'H.No. 838', 'Bara Tooti', 'Haridwar', 'Uttarakhand', 110006);
exec CreateNewUser('3481', 54, 'aaina.choudhary493@gmail.com', 'cQoXsf2o3g', 'Aaina Choudhary', '94', 'Tlaingnuam', 'Mamit', 'Mizoram', 796015);
exec CreateNewUser('3857', 58, 'zoya.virk1808@yahoo.com', 'kxwnQrS7RP', 'Zoya Virk', 'H.No. 265', 'Mahendru', 'Bhagalpur', 'Bihar', 800006);
exec CreateNewUser('4187', 33, 'ranbir.brahmbhatt434@hyderabad.bits-p

In [11]:
def get_property(pid):
    rent = random.randrange(20000, 60000, 1000)
    rentHike = round(random.uniform(0, 100), 4)
    floors = random.randint(1, 3)
    constructionYear = random.randint(2000, 2015)
    startDate = (date(constructionYear, random.randint(1, 12),
                      random.randint(1, 27))).strftime('%d-%b-%Y')
    endDate = (
        date(constructionYear, random.randint(1, 12), random.randint(
            1, 27)) +
        timedelta(days=365 * random.randint(2, 7))).strftime('%d-%b-%Y')
    plinth = round(random.uniform(300, 450), 4)
    aadharID = random.choice(aadhars[:10])
    local = get_random_locality(np.random.choice(locality.state.unique()[:6]))
    lclty = local[1]
    address = fake.street_address().replace('\n', ' ') + ' ' + local[1] + ', ' + local[2] + ' ' + str(local[0])
    
    query = "exec InsertPropertyRecord({}, {}, {}, {}, {}, '{}', '{}', '{}', {}, '{}', '{}');".format(
    pid, rent, rentHike, floors, constructionYear,
    address, startDate, endDate, plinth, lclty,
    aadharID)
    return (query, pid)


In [12]:
properties = []
f = open('property.sql', 'w+')
for pid in range(100, 150):
    prop = get_property(pid)
    print(prop[0])
    properties.append(prop[1])
    f.writelines(prop[0] + '\n')

exec InsertPropertyRecord(100, 47000, 18.3488, 2, 2007, 'H.No. 32 Koshy Circle Dasna Gate, Uttar Pradesh 201009', '15-Feb-2007', '03-Sep-2009', 397.8217, 'Dasna Gate', '4633');
exec InsertPropertyRecord(101, 25000, 92.6367, 1, 2005, 'H.No. 122, Raju Marg Faridabad, haryana 121002', '16-Jul-2005', '06-Aug-2010', 435.3664, 'Faridabad', '2026');
exec InsertPropertyRecord(102, 20000, 98.5309, 2, 2014, '69, Lalla Chowk Chamkaur Sahib, Punjab 140112', '14-May-2014', '22-Dec-2020', 399.2807, 'Chamkaur Sahib', '2131');
exec InsertPropertyRecord(103, 32000, 29.6708, 1, 2001, '01/84 Bhalla Path Chamkaur Sahib, Punjab 140112', '11-Dec-2001', '01-Jan-2007', 371.5183, 'Chamkaur Sahib', '4633');
exec InsertPropertyRecord(104, 23000, 96.0779, 1, 2005, '27 Sen Nagar Dundahera, Haryana 122016', '20-Feb-2005', '22-Feb-2008', 360.5663, 'Dundahera', '3482');
exec InsertPropertyRecord(105, 57000, 59.4519, 3, 2002, '28/14 Ramanathan Circle Bara Tooti, New Delhi 110006', '22-Jul-2002', '17-Oct-2008', 347.455

# Commercial or Residential

In [13]:
random.shuffle(properties)

In [14]:
for com in properties[:25]:
    query = 'INSERT INTO Commercial VALUES({}, {});'.format(com, round(random.uniform(0, 100), 3))
    print(query)
    f.writelines(query + '\n')

INSERT INTO Commercial VALUES(146, 26.465);
INSERT INTO Commercial VALUES(127, 8.119);
INSERT INTO Commercial VALUES(122, 23.987);
INSERT INTO Commercial VALUES(105, 55.199);
INSERT INTO Commercial VALUES(149, 15.653);
INSERT INTO Commercial VALUES(135, 40.852);
INSERT INTO Commercial VALUES(110, 68.937);
INSERT INTO Commercial VALUES(126, 46.991);
INSERT INTO Commercial VALUES(114, 3.264);
INSERT INTO Commercial VALUES(123, 28.81);
INSERT INTO Commercial VALUES(137, 28.272);
INSERT INTO Commercial VALUES(134, 85.961);
INSERT INTO Commercial VALUES(145, 7.118);
INSERT INTO Commercial VALUES(148, 23.342);
INSERT INTO Commercial VALUES(112, 26.455);
INSERT INTO Commercial VALUES(119, 79.125);
INSERT INTO Commercial VALUES(113, 58.979);
INSERT INTO Commercial VALUES(117, 80.391);
INSERT INTO Commercial VALUES(143, 19.782);
INSERT INTO Commercial VALUES(133, 11.478);
INSERT INTO Commercial VALUES(129, 22.481);
INSERT INTO Commercial VALUES(109, 14.899);
INSERT INTO Commercial VALUES(139, 2

In [15]:
for res in properties[25:]:
    query = 'INSERT INTO Residential VALUES({}, {});'.format(res, random.randint(1, 5))
    print(query)
    f.writelines(query + '\n')

INSERT INTO Residential VALUES(101, 3);
INSERT INTO Residential VALUES(138, 5);
INSERT INTO Residential VALUES(120, 5);
INSERT INTO Residential VALUES(142, 3);
INSERT INTO Residential VALUES(102, 4);
INSERT INTO Residential VALUES(115, 1);
INSERT INTO Residential VALUES(125, 4);
INSERT INTO Residential VALUES(104, 3);
INSERT INTO Residential VALUES(107, 4);
INSERT INTO Residential VALUES(147, 3);
INSERT INTO Residential VALUES(116, 5);
INSERT INTO Residential VALUES(103, 5);
INSERT INTO Residential VALUES(111, 4);
INSERT INTO Residential VALUES(121, 4);
INSERT INTO Residential VALUES(128, 1);
INSERT INTO Residential VALUES(141, 5);
INSERT INTO Residential VALUES(130, 1);
INSERT INTO Residential VALUES(118, 4);
INSERT INTO Residential VALUES(136, 3);
INSERT INTO Residential VALUES(100, 5);
INSERT INTO Residential VALUES(106, 3);
INSERT INTO Residential VALUES(131, 1);
INSERT INTO Residential VALUES(140, 1);
INSERT INTO Residential VALUES(108, 2);
INSERT INTO Residential VALUES(144, 5);


# Tenant

In [16]:
for i in range(20):
    query = "exec AddTenant({}, '{}', {}, '{}', '{}');".format(
        random.choice(properties),
        random.choice(aadhars),
        random.randrange(1000, 8000, 200),
        (date(2010, random.randint(1, 12),
                      random.randint(1, 27))).strftime('%d-%b-%Y'),
        (date(2010, random.randint(1, 12), random.randint(
            1, 27)) + timedelta(days=365 * random.randint(2, 7))).strftime('%d-%b-%Y')
    )
    print(query)
    f.writelines(query + '\n')

exec AddTenant(111, '2102', 4400, '02-Oct-2010', '14-Mar-2016');
exec AddTenant(130, '2350', 4400, '19-Mar-2010', '20-Jul-2015');
exec AddTenant(135, '4484', 5400, '11-Jul-2010', '21-Jun-2012');
exec AddTenant(137, '2260', 6200, '16-Dec-2010', '21-May-2015');
exec AddTenant(108, '2128', 1400, '03-Aug-2010', '08-Jun-2014');
exec AddTenant(126, '2435', 7400, '22-Jan-2010', '14-Sep-2015');
exec AddTenant(105, '4187', 7600, '20-Jun-2010', '20-Aug-2015');
exec AddTenant(108, '2383', 3600, '18-May-2010', '09-Mar-2015');
exec AddTenant(100, '4484', 2400, '21-Jan-2010', '25-Oct-2013');
exec AddTenant(106, '3857', 4800, '01-Sep-2010', '13-Sep-2012');
exec AddTenant(112, '3408', 6800, '21-Feb-2010', '14-Mar-2017');
exec AddTenant(143, '2920', 4400, '27-Jul-2010', '15-Aug-2013');
exec AddTenant(102, '2128', 2800, '07-Jul-2010', '15-Oct-2017');
exec AddTenant(114, '2026', 4400, '11-Jul-2010', '08-Sep-2012');
exec AddTenant(143, '3482', 7200, '15-Mar-2010', '15-Sep-2014');
exec AddTenant(109, '2128

# Phone Number

In [17]:
for i in range(35):
    no = fake.phone_number()
    query = "INSERT INTO PHONE VALUES('{}', {});".format(random.choice(aadhars), no)
    print(query)
    f.writelines(query + '\n')

INSERT INTO PHONE VALUES('2026', 6088356159);
INSERT INTO PHONE VALUES('4484', 01484656482);
INSERT INTO PHONE VALUES('4633', +916629946804);
INSERT INTO PHONE VALUES('2260', 03699577738);
INSERT INTO PHONE VALUES('4676', 02148951343);
INSERT INTO PHONE VALUES('2435', +912003791769);
INSERT INTO PHONE VALUES('2026', +916763201632);
INSERT INTO PHONE VALUES('2260', 8708317278);
INSERT INTO PHONE VALUES('2128', 9579868727);
INSERT INTO PHONE VALUES('2435', 04348734714);
INSERT INTO PHONE VALUES('2260', +914558122362);
INSERT INTO PHONE VALUES('4484', 3166587603);
INSERT INTO PHONE VALUES('2128', 06909670546);
INSERT INTO PHONE VALUES('2383', 08893734670);
INSERT INTO PHONE VALUES('4676', 05627298069);
INSERT INTO PHONE VALUES('2026', 0162720465);
INSERT INTO PHONE VALUES('2131', +917556464170);
INSERT INTO PHONE VALUES('3482', 8053100330);
INSERT INTO PHONE VALUES('2131', 2327193745);
INSERT INTO PHONE VALUES('2026', +919912419049);
INSERT INTO PHONE VALUES('3327', 6631931491);
INSERT IN

# Facilities

In [18]:
facilities = ['Gated compound',
'Covered car parking',
'Electricity back up',
'Rainwater harvesting',
'RO water',
'Security system',
'Electrical fitting',
'Kitchen fitting',
'Play area for kids',
'Balconies',
'Gymnasium',
'Sports facilities',
'Clubhouse']

for prop in properties[25:]:
    for fac in random.sample(facilities, random.randint(0, len(facilities))):
        query = "INSERT INTO FACILITY VALUES({}, '{}');".format(
            prop, fac
        )
        print(query)
        f.writelines(query + '\n')

INSERT INTO FACILITY VALUES(101, 'Play area for kids');
INSERT INTO FACILITY VALUES(101, 'Electrical fitting');
INSERT INTO FACILITY VALUES(101, 'Balconies');
INSERT INTO FACILITY VALUES(138, 'Clubhouse');
INSERT INTO FACILITY VALUES(138, 'Electricity back up');
INSERT INTO FACILITY VALUES(138, 'Sports facilities');
INSERT INTO FACILITY VALUES(138, 'Gymnasium');
INSERT INTO FACILITY VALUES(138, 'Covered car parking');
INSERT INTO FACILITY VALUES(138, 'Electrical fitting');
INSERT INTO FACILITY VALUES(138, 'RO water');
INSERT INTO FACILITY VALUES(138, 'Security system');
INSERT INTO FACILITY VALUES(138, 'Play area for kids');
INSERT INTO FACILITY VALUES(138, 'Rainwater harvesting');
INSERT INTO FACILITY VALUES(120, 'Electricity back up');
INSERT INTO FACILITY VALUES(120, 'Rainwater harvesting');
INSERT INTO FACILITY VALUES(120, 'Kitchen fitting');
INSERT INTO FACILITY VALUES(120, 'RO water');
INSERT INTO FACILITY VALUES(120, 'Gymnasium');
INSERT INTO FACILITY VALUES(120, 'Balconies');
I

In [19]:
f.close()